In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini Bracket Generation

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Falokpattani%2Fgenai-brackets%2Fmain%2Fgemini_bracket_generation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/alokpattani/genai-brackets/main/gemini_bracket_generation.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/alokpattani/genai-brackets/blob/main/gemini_bracket_generation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

## Setup
---

In [ ]:
#@title Install/Upgrade Relevant Packages
!pip install --upgrade -q google-genai
!pip install -q plotly
!pip install -q kaleido
!pip install -q weasyprint

In [ ]:
#@title If in Colab (non-Enterprise), Run This Cell to Authenticate
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
#@title Imports
import os

from google import genai
from google.genai import types
import base64

import pandas as pd
import numpy as np
import math

import plotly.graph_objects as go
import plotly.io as pio

import json

import mimetypes
from urllib.parse import urlparse

import typing
import copy

import kaleido

import IPython.display
from IPython.display import HTML, Markdown, display, clear_output
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

import markdown
from weasyprint import HTML
from google.colab import files

In [ ]:
#@title Set Up Cloud Project, GenAI Client, and Models
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

GENAI_CLIENT = genai.Client(
  vertexai=True,
  project=PROJECT_ID,
  location=LOCATION
  )

GEMINI_MODEL_ID = "gemini-2.0-flash-001" # @param {type: "string"}

IMAGEN_MODEL_ID = "imagen-3.0-generate-002" # @param {type: "string"}

In [ ]:
#@title Set Bracket Theme, # of Items, and Other Parameters
BRACKET_THEME = "Ben & Jerry's Ice Cream Flavors" #@param{type:"string"}

NUM_ITEMS = 64 #@param{type:"raw"} [16, 32, 64]

# Eventually add functionality for different region #s (only allow 4 for now)
NUM_REGIONS = 4 #@param {type: "raw"} [4]

GROUNDING_SOURCE = "Provided Material" #@param{type:"string"} ["None", "Google Search", "Provided Material"]

PROVIDED_MATERIAL_FILE_URI = "gs://genai_public/brackets/grounding_sources/2024_ben_and_jerrys_flavors.png" #@param{type:"string"}

BRACKET_ADVANCEMENT_SPECIAL_CRITERIA = "cookie dough is the best" #@param{type:"string"}

# Cuts out white space above/below actual bracket content in HTML/image output
# Potentially useful for 16 & 32 items, where result is wider than it is tall
# False will lead to 8.5x11 "landscape"-size bracket HTML/image output
TRIM_BRACKET_VISUAL_VERTICALLY = True #@param{type:"boolean"}

NUM_ROUNDS = math.ceil(math.log2(NUM_ITEMS))

RANKS = [str(i) for i in range(1, NUM_ITEMS + 1)]

NUM_SEEDS_PER_REGION = NUM_ITEMS // NUM_REGIONS

REGION_NUMS = [str(i) for i in range(1, NUM_REGIONS + 1)]

SEEDS = [str(i) for i in range(1, NUM_SEEDS_PER_REGION + 1)]

BRACKET_TITLE = f"{BRACKET_THEME} Bracket from Gemini"

BRACKET_FOOTER = (f"{BRACKET_THEME} and rankings for bracket generated by "
  "Gemini, with advancement determined by Gemini with preference for item seed"
  f'''{(" and special criteria of '" + BRACKET_ADVANCEMENT_SPECIAL_CRITERIA +
    "'") if BRACKET_ADVANCEMENT_SPECIAL_CRITERIA != "" else ""}.'''
  "<br>See more at goo.gle/gemini-bracket-code."
  )

BRACKET_DESCRIPTION = f'''{BRACKET_TITLE}:
- {NUM_ITEMS} {BRACKET_THEME} divided into {NUM_REGIONS} regions
- {NUM_SEEDS_PER_REGION} seeds per region with regions {REGION_NUMS}
- seeds {SEEDS} in each region
- {NUM_ROUNDS} rounds
- grounding source: {GROUNDING_SOURCE}
{(('- provided grounding material: ' + PROVIDED_MATERIAL_FILE_URI)
  if GROUNDING_SOURCE == 'Provided Material' else '')}

{BRACKET_FOOTER}
'''

print(BRACKET_DESCRIPTION)

In [ ]:
#@title Various Helpful Functions
# Function to wrap text after certain # of characters to use on bracket
def wrap_text(text, max_chars):
    words = text.split()
    wrapped_text = ""
    current_line = ""
    for word in words:
        if len(current_line) == 0:
            current_line = word
        elif len(current_line) + len(word) + 1 <= max_chars:
            current_line += " " + word
        else:
            wrapped_text += current_line + "<br>"
            current_line = word
    wrapped_text += current_line
    return wrapped_text

# Function to get mime type from GCS URI
def get_mime_type_from_gcs_uri(file_uri):
  """Gets the MIME type from a GCS file URI.

  Args:
    file_uri: The GCS file URI.

  Returns:
    The MIME type of the file.
  """
  parsed_url = urlparse(file_uri)
  file_path = parsed_url.path.lstrip('/')
  mime_type, _ = mimetypes.guess_type(file_path)
  return mime_type

# Function to help display images in notebook
def display_image(
    image,
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)

# Function to encode image
def get_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode()

In [ ]:
#@title Other Clerical Stuff

# Expand max column width when displaying data frames to handle longer text
pd.set_option('display.max_colwidth', 200)

## Bracket Visual Creation
---

In [ ]:
#@title Create Table with All Possible Round Names Up to 7-Round Bracket
BRACKET_ROUNDS = pd.DataFrame({
  'round_desc': list(range(7, -1, -1)),
  'round_name': ['Round of 128', 'Round of 64', 'Round of 32',
    'Round of 16', 'Quarterfinals', 'Semifinals', 'Final', 'Champion']
  })

BRACKET_ROUNDS['num_items_in_round'] = 2**BRACKET_ROUNDS['round_desc']

BRACKET_ROUNDS['num_matchups_in_round'] = np.where(
  BRACKET_ROUNDS['round_desc']== 0,
  0,
  BRACKET_ROUNDS['num_items_in_round'] // 2
  )

In [ ]:
#@title Functions to Create Brackets with Various Pieces
def create_df_to_represent_bracket(num_items, round_nums_to_names_df):

    num_rounds = math.ceil(math.log2(num_items))
    # Calculate the number of slots in each round
    slots_per_round = []

    # Create a list to store all the rows for the DataFrame
    all_rows = []

    # Iterate through the rounds and generate rows for each slot
    for round_num in range(num_rounds+1):
      slots_per_round.append(int(num_items / (2**round_num)))
      for slot_num in range(slots_per_round[round_num]):
          row = {
            'round_asc': round_num + 1,
            'round_desc': num_rounds - round_num,
            'round_slot': slot_num + 1
            }
          all_rows.append(row)

    # Create dataframe from list of rows, merge in round names
    bracket_df = pd.merge(
      round_nums_to_names_df,
      pd.DataFrame(all_rows),
      on = ['round_desc'],
      how = 'right'
      )

    return bracket_df


def get_slot_seed_ordering_df(num_seeds = 16):
  closest_power_of_2 = 2**(math.ceil(math.log2(num_seeds)))

  if(closest_power_of_2 == 2):
    seeds_power2 = [1, 2]

  elif(closest_power_of_2 == 4):
    seeds_power2 = [1, 4, 3, 2]

  elif(closest_power_of_2 == 8):
    seeds_power2 = [1, 8, 4, 5, 3, 6, 2, 7]

  elif(closest_power_of_2 == 16):
    seeds_power2 = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

  final_seeds_order = [(seed if seed <= num_seeds else pd.NA)
    for seed in seeds_power2]

  slot_seed_ordering_df = pd.DataFrame({
    'slot': range(1, closest_power_of_2 + 1),
    'seed': final_seeds_order
    })

  return(slot_seed_ordering_df)


def get_bracket_viz_config(num_items):
    """Returns a dictionary containing bracket configuration constants."""

    num_rounds = math.ceil(math.log2(num_items))

    config = {
        "bracket_line_color": 'black',
        "bracket_line_width": 1,
        "single_rect_width": 122,
        "single_rect_height": 22,
        "between_round_space_width": 14,
        "between_rect_space_height": 15,
        "num_rounds": num_rounds,
        "round0_rect_inflation_factor": 1.25,
        "round_name_space_vs_max_y": 12,
        "round_name_height": 30,
        "round_name_font_size": 18,
        "title_space_vs_max_round_name_y": 16,
        "title_font_size": 30,
        "footer_space_vs_min_bracket_rect_y": 16,
        "footer_font_size": 14,
        "vertical_margin_space": 15,
        "overall_width": (num_rounds + 1) * 260,
        "image_space_vs_min_y": 12,
        "image_width": 8 * (num_rounds**2) - 28,
        "image_height": 8 * (num_rounds**2) - 28
        }

    config["overall_height"] = config["overall_width"] * 8.5 / 11
    config["left_end"] = -config["overall_width"] / 2
    config["right_end"] = config["overall_width"] / 2
    config["top"] = config["overall_height"] / 2
    config["bottom"] = -config["overall_height"] / 2

    return config


def create_empty_figure_for_bracket(num_items):
    """Creates an empty Plotly bracket figure using provided configuration."""
    BrCo = get_bracket_viz_config(num_items)

    bracket_layout = go.Figure()

    axis_dict = dict(title=None, showticklabels=False, showgrid=False,
      zeroline=False, scaleratio=1)

    bracket_layout.update_layout(
      margin=dict(t=0, r=0, b=0, l=0),
      autosize=False,
      width=BrCo["overall_width"],
      height=BrCo["overall_height"],
      xaxis=axis_dict,
      yaxis=axis_dict,
      plot_bgcolor="white",
      showlegend=False,
      )

    bracket_layout.update_xaxes(range=[BrCo["left_end"], BrCo["right_end"]])
    bracket_layout.update_yaxes(range=[BrCo["bottom"], BrCo["top"]])

    return bracket_layout


def get_bracket_rect_top_y_by_slot_num(slot_num, num_items_in_round,
    round_vs_ovr_num, between_rect_space, rect_height,
    connected_matchups):

    rank = (slot_num-1) % (num_items_in_round/2) + 1

    num_spaces = np.where(connected_matchups,
      (num_items_in_round/4 + 1) - ((rank + 1) // 2) * 2,
      ((num_items_in_round/2 - (2 * rank) + 1) *
          (2 **(round_vs_ovr_num - 1)))
      )

    num_rects = np.where(connected_matchups,
      (num_items_in_round/4) - rank + 1,
      num_spaces + 0.5
      )

    rect_top_y = np.where(
      (num_items_in_round <= 2), rect_height / 2,
      (num_spaces * between_rect_space) + (num_rects * rect_height)
      )

    return(rect_top_y)


def get_bracket_visual_element_dfs_from_bracket_df(num_items, start_bracket_df):
    BrCo = get_bracket_viz_config(num_items)

    num_rounds = math.ceil(math.log2(num_items))

    bracket_df = start_bracket_df.copy()

    bracket_df['rect_width'] = (BrCo['single_rect_width'] *
      np.where(bracket_df['round_desc'] == 0,
        BrCo['round0_rect_inflation_factor'],
        1)
      )

    bracket_df['rect_height'] = (BrCo['single_rect_height'] *
      np.where(bracket_df['round_desc'] == 0,
        BrCo['round0_rect_inflation_factor'],
        1)
      )

    bracket_df['round_width_offset'] = np.where(bracket_df['round_desc'] == 0,
      0,
      ((bracket_df['round_desc'] - 1) *
        (BrCo['between_round_space_width'] + bracket_df['rect_width'])
        +
        BrCo['single_rect_width'] * BrCo['round0_rect_inflation_factor']
        )
      )

    bracket_df['round_space_height'] = (bracket_df['round_asc'] *
      (BrCo['between_rect_space_height'] + BrCo['single_rect_height']))

    bracket_df['rect_left_x'] = np.where(bracket_df['round_slot'] <=
      (bracket_df['num_items_in_round'] // 2),
      -bracket_df['round_width_offset'] - bracket_df['rect_width']/2,
      bracket_df['round_width_offset'] - bracket_df['rect_width']/2
      )

    bracket_df['rect_right_x'] = np.where(bracket_df['round_slot'] <=
      (bracket_df['num_items_in_round'] // 2),
      -bracket_df['round_width_offset'] + bracket_df['rect_width']/2,
      bracket_df['round_width_offset'] + bracket_df['rect_width']/2
      )

    bracket_df['rect_mid_x'] = (bracket_df['rect_left_x'] +
      bracket_df['rect_right_x']) / 2

    bracket_df['rect_top_y'] = get_bracket_rect_top_y_by_slot_num(
      slot_num = bracket_df['round_slot'],
      num_items_in_round = bracket_df['num_items_in_round'],
      round_vs_ovr_num = (num_rounds - bracket_df['round_desc']
        ).astype('float'),
      between_rect_space = BrCo['between_rect_space_height'],
      rect_height = bracket_df['rect_height'],
      # connected_matchups = False,
      # Only connect matchups for 1st round for now
      connected_matchups = np.where(num_rounds == bracket_df['round_desc'],
        True, False)
      )

    bracket_df['rect_bottom_y'] = (bracket_df['rect_top_y'] -
      bracket_df['rect_height'])

    bracket_df['rect_mid_y'] = (bracket_df['rect_top_y'] +
      bracket_df['rect_bottom_y']) / 2

    # Get next round info
    bracket_df['next_round_desc'] = np.where(bracket_df['round_desc'] == 0,
      pd.NA, bracket_df['round_desc'] - 1)

    # Next round slot is grouping pairs - divide current round by 2 & round up
    bracket_df['next_round_slot'] = np.where(bracket_df['round_desc'] == 0,
      pd.NA, np.ceil(bracket_df['round_slot'] / 2).astype(int))

    next_round_cols = ['round_desc', 'round_slot', 'rect_left_x',
      'rect_right_x', 'rect_mid_x', 'rect_top_y', 'rect_bottom_y', 'rect_mid_y']

    # Self-join to get next round rectangle info to help draw connectors
    bracket_df = pd.merge(
      bracket_df,
      bracket_df[next_round_cols].rename(
        columns={col: 'next_' + col for col in next_round_cols}
        ),
      on = ['next_round_desc', 'next_round_slot'],
      how = "left"
      )

    bracket_df['next_round_horiz_dir'] = np.where(
      bracket_df['round_desc'] == 0,
      'NA',
      np.where(
        bracket_df['rect_mid_x'] < bracket_df['next_rect_mid_x'],
        'right',
        np.where(
          bracket_df['rect_left_x'] > bracket_df['next_rect_left_x'],
          'left',
          'same'
          )
        )
      )

    bracket_df['rect_outgoing_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['next_round_horiz_dir'] == 'left',
        bracket_df['rect_left_x'],
        np.where(
          bracket_df['next_round_horiz_dir'] == 'right',
          bracket_df['rect_right_x'],
          # Placeholder for "same" (not clear what it should be)
          bracket_df['rect_mid_x']
          )
        )
      )

    bracket_df['next_rect_incoming_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['next_round_horiz_dir'] == 'left',
        bracket_df['next_rect_right_x'],
        np.where(
          bracket_df['next_round_horiz_dir'] == 'right',
          bracket_df['next_rect_left_x'],
          # Placeholder for "same"
          bracket_df['next_rect_mid_x']
          )
        )
      )

    bracket_df['halfway_to_next_round_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      (bracket_df['rect_outgoing_x'] + bracket_df['next_rect_incoming_x']) / 2
      )

    bracket_df['next_round_vert_dir'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['rect_mid_y'] < bracket_df['next_rect_mid_y'],
        'up',
        np.where(
          bracket_df['rect_top_y'] > bracket_df['next_rect_top_y'],
          'down',
          'same'
          )
        )
      )

    round_names_df = (bracket_df.
      assign(
        next_round_horiz_dir = lambda x: np.where(
          x['round_desc'] == 0,
          "NA",
          x['next_round_horiz_dir']
          )
        ).
        groupby(['round_desc', 'round_name', 'next_round_horiz_dir']).
        agg(
          round_text_left_x=('rect_left_x', 'min'),
          round_text_right_x=('rect_right_x', 'max'),
          round_rect_top_y=('rect_top_y', 'max'),
          round_rect_bottom_y=('rect_bottom_y', 'min')
          ).
        reset_index()
      )

    round_names_df['round_text_mid_x'] = (round_names_df['round_text_left_x'] +
      round_names_df['round_text_right_x']) / 2

    round_names_df['round_name_top_y'] = np.where(
      round_names_df['round_desc'] == 0,
      round_names_df['round_rect_top_y'],
      max(round_names_df['round_rect_top_y'])
      ) + BrCo['round_name_space_vs_max_y'] + BrCo['round_name_height']

    round_names_df['round_name_font_size'] = BrCo['round_name_font_size']

    bracket_and_round_names_dfs_dict = dict({"bracket_df": bracket_df,
      "round_names_df": round_names_df})

    return(bracket_and_round_names_dfs_dict)


def build_bracket_viz_from_dfs(num_items, bracket_df, round_names_df,
    bracket_title = None, bracket_footer = None,
    trim_bracket_viz_vertically = False
    ):
    BrCo = get_bracket_viz_config(num_items)

    bracket_viz = create_empty_figure_for_bracket(num_items)

    for index, row in bracket_df.iterrows():
        # Add bracket rectangles for each slot, 1 by 1
        bracket_viz.add_shape(
          dict(
            type="rect",
            xref="x",
            yref="y",
            x0=row['rect_left_x'],
            y0=row['rect_top_y'],
            x1=row['rect_right_x'],
            y1=row['rect_bottom_y'],
            line_color=BrCo['bracket_line_color'],
            line_width=BrCo['bracket_line_width']
            )
          )

        # Add connectors for all rounds except 0
        if(row['round_desc'] != 0):
          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['rect_outgoing_x'],
              y0=row['rect_mid_y'],
              x1=row['halfway_to_next_round_x'],
              y1=row['rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['halfway_to_next_round_x'],
              y0=row['rect_mid_y'],
              x1=row['halfway_to_next_round_x'],
              y1=row['next_rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['halfway_to_next_round_x'],
              y0=row['next_rect_mid_y'],
              x1=row['next_rect_incoming_x'],
              y1=row['next_rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

    # Add each round name above corresponding bracket slot column(s)
    for index, row in round_names_df.iterrows():
        # Add round name above each "column" of brackets
        bracket_viz.add_annotation(
          dict(
            x=row['round_text_mid_x'],
            y=row['round_name_top_y'],
            text=row['round_name'],
            font=dict(size = row['round_name_font_size']),
            showarrow=False,
            xanchor="center",
            yanchor="top"
            )
          )

    # Add title to bracket
    if(bracket_title is not None and bracket_title != ""):
      # Take average of furthest round name text x-coordinates on either side
      title_text_mid_x = (round_names_df['round_text_left_x'].min() +
        round_names_df['round_text_right_x'].max()) / 2

      # Take max of round name text y-coordinates and add spacing + font size
      title_text_top_y = (round_names_df['round_name_top_y'].max() +
        BrCo['title_space_vs_max_round_name_y']) + BrCo['title_font_size']

      bracket_viz.add_annotation(
        dict(
          x=title_text_mid_x,
          y=title_text_top_y,
          text=bracket_title,
          font=dict(size = BrCo['title_font_size']),
          showarrow=False,
          xanchor="center",
          yanchor="top"
          )
        )

    # Add footer to bracket
    if(bracket_footer is not None and bracket_footer != ""):
      # Take average of furthest round name text x-coordinates on either side
      footer_text_mid_x = (round_names_df['round_text_left_x'].min() +
        round_names_df['round_text_right_x'].max()) / 2

      footer_text_max_length = (round_names_df['round_text_right_x'].max() -
        round_names_df['round_text_left_x'].min()) - 200

      # Take min of bracket df text y-coordinates and subtract spacing
      footer_text_top_y = (bracket_df['rect_bottom_y'].min() -
        BrCo['footer_space_vs_min_bracket_rect_y'])

      # Wrap bracket footer if very long
      bracket_footer_wrapped = wrap_text(bracket_footer, 150)

      bracket_viz.add_annotation(
        dict(
          x=footer_text_mid_x,
          y=footer_text_top_y,
          text=bracket_footer_wrapped,
          font=dict(size = BrCo['footer_font_size']),
          showarrow=False,
          xanchor="center",
          yanchor="top"
          )
        )

    return bracket_viz

def add_bracket_items_to_bracket_viz(bracket_viz, bracket_df, bracket_items_df,
    max_display_char_width = 15):

    bracket_items_df['chunk_display_text'] = (bracket_items_df.
      apply(lambda row:
        wrap_text(f"{row['display_text']}", max_display_char_width),
        axis=1
        )
      )

    bracket_items_for_display = pd.merge(
      bracket_items_df,
      bracket_df,
      on = ["round_desc", "round_slot"],
      how = "left"
      )

    bracket_items_for_display['display_text_font_size'] = (
      bracket_items_for_display['rect_width'] /
      (bracket_items_for_display['chunk_display_text'].str.count('<br>') + 1)
      ) * 0.12

    bracket_viz_with_items = copy.deepcopy(bracket_viz)

    for index, row in bracket_items_for_display.iterrows():
        bracket_viz_with_items.add_annotation(
          dict(
            x=row['rect_left_x'],
            y=row['rect_top_y'],
            text=row['chunk_display_text'],
            font=dict(size = row['display_text_font_size']),
            hovertext=row['hover_text'],
            showarrow=False,
            xanchor="left",
            yanchor="top"
            )
          )

    return bracket_viz_with_items

def build_bracket_viz(num_items, round_nums_to_names_df, bracket_title = None,
    bracket_footer = None, bracket_items_df = None, existing_bracket_viz = None,
    trim_bracket_viz_vertically = False
    ):

    BrCo = get_bracket_viz_config(num_items)

    initial_bracket_df = create_df_to_represent_bracket(num_items,
      round_nums_to_names_df)

    bracket_dfs = get_bracket_visual_element_dfs_from_bracket_df(
      num_items, initial_bracket_df)

    if(existing_bracket_viz is None or not existing_bracket_viz.layout):
      bracket_viz = build_bracket_viz_from_dfs(
          num_items,
          bracket_dfs['bracket_df'],
          bracket_dfs['round_names_df'],
          bracket_title,
          bracket_footer
          )
    else:
      bracket_viz = existing_bracket_viz

    if(bracket_items_df is not None and not bracket_items_df.empty):
      bracket_viz_with_items = add_bracket_items_to_bracket_viz(
          bracket_viz,
          bracket_dfs['bracket_df'],
          bracket_items_df
          )

    else:
      bracket_viz_with_items = copy.deepcopy(bracket_viz)

    if(trim_bracket_viz_vertically):
      # Extract annotations into df
      annotation_list = []

      for annotation in bracket_viz_with_items.layout.annotations:
        annotation_dict = annotation.to_plotly_json()
        annotation_list.append(annotation_dict)

      annotation_df = pd.DataFrame(annotation_list)

      annotation_df['font_size'] = [font['size'] for font in
        annotation_df['font']]

      annotation_df['text_num_lines'] = (annotation_df['text'].str.count('<br>')
        + 1)

      # Find top / bottom text position of each annotation
      annotation_df['top_y'] = (annotation_df['y'] +
        annotation_df['font_size'] * annotation_df['text_num_lines'] *
        np.where(annotation_df['yanchor'] == "top", 0,
          np.where(annotation_df['yanchor'] == "bottom", 1, 0.5)
          )
        )

      annotation_df['bottom_y'] = (annotation_df['y'] -
        annotation_df['font_size'] * annotation_df['text_num_lines'] *
        np.where(annotation_df['yanchor'] == "top", 1,
          np.where(annotation_df['yanchor'] == "bottom", 0, 0.5)
          )
        )

      # Find highest and lowest annotations
      highest_annotation = annotation_df['top_y'].max()
      lowest_annotation = annotation_df['bottom_y'].min()

      # Adjust y-axis range and height according to bracket text extremes
      bracket_viz_with_items.update_layout(
        yaxis_range = [lowest_annotation - BrCo["vertical_margin_space"],
          highest_annotation + BrCo["vertical_margin_space"]],
        height = highest_annotation - lowest_annotation +
          BrCo["vertical_margin_space"] * 2
        )

    # Create dict of initial brackets, bracket_dfs, and bracket_viz to return
    bracket_viz_and_dfs_dict = dict({
      "initial_bracket_df": initial_bracket_df,
      "bracket_dfs": bracket_dfs,
      "bracket_viz": bracket_viz_with_items
      })

    return bracket_viz_and_dfs_dict

In [ ]:
#@title Create Initial Bracket Shell
empty_bracket_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_items_df = None,
  existing_bracket_viz = None,
  trim_bracket_viz_vertically = TRIM_BRACKET_VISUAL_VERTICALLY
  )

empty_bracket_viz = empty_bracket_viz_and_dfs['bracket_viz']

display(empty_bracket_viz)

## Gemini-Based Ranked List Generation
---

In [ ]:
#@title Functions to Get Ranked Item List & Extract Using Controlled Generation
def get_list_of_items_that_fit_bracket_theme(num_items, bracket_theme,
    grounding_source, provided_material_file_uri):

    list_generation_system_instruction = types.Part.from_text(text="""
    You are an expert rankings list creator, who can create rankings lists of
    various sizes given a specific theme.

    If Google Search access is enabled, use search results in creating the list.

    If files are provided with items and information relevant to the theme of
    the list, use information directly in the files in creating the list.

    Make sure that each item returned is real, has a unique name (no duplicates
    - add text to clarify if items are similar), and fits within the list theme.

    Rank the items based on the prevalence, popularity, quality, or otherwise
    superiority of the items (with criteria depending on the theme of the
    bracket).

    Provide a description of each item including an explanation of why it was
    ranked where it was, including all items through the end of the ranking.

    Only return the list of items in ranked order along with the description or
    explanation, no other extra information.
    """)

    list_generation_tools = ([types.Tool(google_search=types.GoogleSearch())] if
      grounding_source == "Google Search" else [])

    list_generation_config = types.GenerateContentConfig(
      temperature = 1,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [list_generation_system_instruction],
      response_modalities = ["TEXT"],
      tools = list_generation_tools
      )

    list_generation_prompt = types.Part.from_text(text=
    f"Create a ranked list of {num_items} {bracket_theme}.")

    content_generation_parts = [list_generation_prompt]

    if (grounding_source == "Provided Material" and provided_material_file_uri
        and provided_material_file_uri != ""):
        list_generation_ref_material = types.Part.from_uri(
          file_uri=provided_material_file_uri,
          mime_type=get_mime_type_from_gcs_uri(provided_material_file_uri)
          )

        content_generation_parts = (content_generation_parts +
          [list_generation_ref_material])

    list_generation_contents = [
      types.Content(
        role="user",
        parts=content_generation_parts
        )
      ]

    list_generation_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=list_generation_contents,
      config=list_generation_config
      )

    list_generation_response_text = list_generation_response.text

    # display(Markdown(list_generation_response_text))

    return(list_generation_response_text)


def get_ranked_list_from_text_response(num_items, list_generation_text_response
    ):
    ranked_list_response_schema = {
      "type": "ARRAY",
      "items": {
        "type": "OBJECT",
        "properties": {
          "item": {
            "type": "STRING"
          },
          "description": {
            "type": "STRING"
          },
          "rank": {
            "type": "STRING",
            "enum": RANKS
          },
          "short_name": {
            "type": "STRING"
          }
        },
        "required": [
          "item",
          "description",
          "rank",
          "short_name"
          ]
      }
    }

    ranked_list_parsing_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        response_modalities = ["TEXT"],
        response_mime_type = "application/json",
        response_schema = ranked_list_response_schema
        )

    ranked_list_generation_prompt = types.Part.from_text(text=f"""
    Parse the provided text containing a ranked list into a JSON object with
    {num_items} elements that adheres to the following schema:
    {ranked_list_response_schema}
    Item, ranking, and description should come directly from the text.
    short_name should be a shorter version of the item name that is 10
    characters or fewer, including spaces (can be an abbreviation). Each item,
    ranking, description, and short_name should be unique - there should no
    duplicates across the list in any field.
    Here is the text with the ranked list:
    {list_generation_text_response}
    """)

    ranked_list_generation_contents = [
      types.Content(
        role="user",
        parts=[
          ranked_list_generation_prompt
        ]
        )
      ]

    ranked_list_generation_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=ranked_list_generation_contents,
      config=ranked_list_parsing_config
      )

    ranked_list_response_json = ranked_list_generation_response.text

    return(ranked_list_response_json)

def get_ranked_df_of_bracket_theme_items(num_items, bracket_theme,
    grounding_source, provided_material_file_uri, num_retries = 10):

    for i in range(num_retries):
      # First get items list in text form
      items_list_for_bracket_text = get_list_of_items_that_fit_bracket_theme(
        num_items = NUM_ITEMS,
        bracket_theme = BRACKET_THEME,
        grounding_source = GROUNDING_SOURCE,
        provided_material_file_uri = PROVIDED_MATERIAL_FILE_URI
        )

      # Then extract ranked list in JSON form
      ranked_list_response_json = get_ranked_list_from_text_response(
          NUM_ITEMS, items_list_for_bracket_text)

      # Convert to DF
      ranked_list_df = (pd.DataFrame(
        json.loads(ranked_list_response_json)).
        astype({
          'rank':'int64'
          })
        )

      # Check that all fields have unique values across all num_items
      all_fields_unique = (ranked_list_df.nunique() == NUM_ITEMS).all()

      if all_fields_unique:
        return ranked_list_df
      else:
        print("All ranking fields not unique, trying to generate again.")

    raise RuntimeError("Failed to generate a valid ranked list of items after "
      f"{num_retries} retries.")

In [ ]:
#@title Call Function to Get Ranked DF of Specific # Items from Bracket Theme

ranked_list_df = get_ranked_df_of_bracket_theme_items(
  num_items = NUM_ITEMS,
  bracket_theme = BRACKET_THEME,
  grounding_source = GROUNDING_SOURCE,
  provided_material_file_uri = PROVIDED_MATERIAL_FILE_URI
  )

display(ranked_list_df)

## Bracket Generation Using Ranked List
---

In [ ]:
#@title Generate Bracket Directly from Ranked List (Using Rules/Logic)
bracket_generation_df = ranked_list_df.copy()

# Divide items into seed by original rank into # of regions
bracket_generation_df['seed'] = ((bracket_generation_df['rank'] - 1) //
  NUM_REGIONS) + 1

# Divide items into region by "S-Curve" logic across seeds / original ranks
bracket_generation_df['region_num'] = np.where(
  # For odd seeds, follow overall rank mod # regions in order
  bracket_generation_df['seed'] % 2 == 1,
  ((bracket_generation_df['rank'] - 1) % NUM_REGIONS) + 1,
  # For even seeds, follow overall rank mod # regions in reverse order
  ((NUM_REGIONS - bracket_generation_df['rank']) % NUM_REGIONS) + 1
  )

bracket_generation_df['region_name'] = ("Region " +
  bracket_generation_df['region_num'].astype(str))

bracket_generation_df.rename(columns = {"rank": "original_rank"},
  inplace = True)

display(bracket_generation_df)

In [ ]:
#@title Show Generated Bracket with Items
bracket_generation_with_info = pd.merge(
  bracket_generation_df,
  (get_slot_seed_ordering_df(NUM_SEEDS_PER_REGION).
    rename(columns={'slot': 'region_slot'})
    ),
  on = "seed",
  how = "left"
  )

bracket_generation_with_info['overall_slot'] = ((
  bracket_generation_with_info['region_num'] - 1) * NUM_SEEDS_PER_REGION +
  bracket_generation_with_info['region_slot'])

bracket_generation_with_info['round_desc'] = NUM_ROUNDS

bracket_generation_with_info['round_slot'] = (
    bracket_generation_with_info['overall_slot'])

bracket_generation_with_info['display_text'] = (bracket_generation_with_info.
  apply(lambda row: f"{row['seed']} {row['short_name']}", axis=1)
  )

bracket_generation_with_info['hover_text'] = (bracket_generation_with_info.
  apply(lambda row:
    f"{row['item']}<br>"
    f"<b>Seed:</b> {row['seed']}<br>"
    f"<b>Region:</b> {row['region_name']}<br>"
    f"<b>Description:</b> {row['description']}<br>"
    f"<b>Original Overall Rank:</b> {row['original_rank']}<br>"
    ,
    axis=1
    )
  )

bracket_with_items_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_generation_with_info,
  empty_bracket_viz,
  trim_bracket_viz_vertically = TRIM_BRACKET_VISUAL_VERTICALLY
  )

bracket_with_items_viz = bracket_with_items_viz_and_dfs['bracket_viz']

display(bracket_with_items_viz)

## Bracket Advancement
---

In [ ]:
#@title Function to Get Single Matchup Winner from Gemini
def get_matchup_winner_from_gemini(matchup_dict, bracket_theme,
    advancement_special_criteria, grounding_source, provided_material_file_uri):

    matchup_winner_system_instruction = types.Part.from_text(text="""
    You are an expert at determining winners in head-to-head matchups in
    brackets - assessing each matchup and deciding who should win based on
    significant criteria related to the bracket theme. If relevant, use the
    reference material provided along with any specified advancement criteria to
    help determine the winner and winning margin (from 1 of 3 categories) as
    well as provide explanations for each matchup.
    """)

    matchup_winner_response_schema = {
      "type": "OBJECT",
      "properties": {
        "winning_item": {
          "type": "STRING",
          # Force it to return 1 of 2 items in matchup
          "enum": [matchup_dict['item_1'], matchup_dict['item_2']],
          "nullable": "False"
          },
        "winning_margin": {
          "type": "STRING",
          "enum": ['Close', 'Comfortable', 'Blowout'],
          "nullable": "False"
          },
        "explanation": {
          "type": "STRING",
          "nullable": "False"
          }
      },
      "required": [
        "winning_item",
        "winning_margin",
        "explanation"
      ]
    }

    matchup_winner_config = types.GenerateContentConfig(
      temperature = 1.0,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [matchup_winner_system_instruction],
      response_modalities = ["TEXT"],
      response_mime_type = "application/json",
      response_schema = matchup_winner_response_schema
      )

    matchup_winner_prompt = types.Part.from_text(text=f"""
    Given the following matchup in a {bracket_theme} bracket, determine the
    winner. Favor these special criteria in deciding the winner of each matchup:
    {advancement_special_criteria}.
    After prioritizing the special criteria, you can generally adhere
    to the seeds (lower seed numbers like 1 and 2 are more likely to win than
    higher seed numbers like 15 or 16), but also allow for some randomness (i.e.
    upsets) in the results as well. Provide a projected winning margin (Close,
    Comfortable, or Blowout) and short explanation (1 sentence) for why the
    winner advanced over the loser.
    Below is information on the matchup:
    {matchup_dict}
    """)

    content_generation_parts = [matchup_winner_prompt]

    if (grounding_source == "Provided Material" and provided_material_file_uri
        and provided_material_file_uri != ""):
        bracket_advancement_ref_material = types.Part.from_uri(
          file_uri=provided_material_file_uri,
          mime_type=get_mime_type_from_gcs_uri(provided_material_file_uri)
          )

        content_generation_parts = (content_generation_parts +
          [bracket_advancement_ref_material])

    matchup_winner_contents = [
      types.Content(
        role="user",
        parts=content_generation_parts
        )
      ]

    matchup_winner_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=matchup_winner_contents,
      config=matchup_winner_config
      )

    matchup_winner_response_json = matchup_winner_response.text

    return(matchup_winner_response_json)

In [ ]:
#@title Get Bracket Matchups and Advancement by Round
original_bracket_df = bracket_generation_with_info.copy()

# Initialize this round to be NUM_ROUNDS
this_round = NUM_ROUNDS

# Initialize list to collect round-by-round-results
round_by_round_results = []

while this_round > 0:
    # Initialize this round bracket df to be copy of original bracket df
    if(this_round == NUM_ROUNDS):
      this_round_bracket_df = original_bracket_df.copy()
    # Otherwise filter original bracket df to teams that advanced in prev round
    else:
      this_round_bracket_df = (original_bracket_df[
        original_bracket_df['item'].isin(prev_round_results['winning_item'])].
        reset_index(drop = True)
        )
      del prev_round_results

    # Set to next round
    this_round_bracket_df['next_round_desc'] = (this_round - 1)

    # Next round slot based on ovr slot & # rounds into bracket (grouping by
    # powers of 2) - helps with determining next round matchups
    this_round_bracket_df['next_round_slot'] = np.ceil(
      this_round_bracket_df['overall_slot'] / 2**(NUM_ROUNDS -
        this_round_bracket_df['next_round_desc'])
      ).astype(int)

    this_round_bracket_df_for_merge = this_round_bracket_df[['item',
      'description', 'original_rank', 'short_name', 'seed', 'round_slot',
      'next_round_desc', 'next_round_slot']]

    this_round_matchups = pd.merge(
      this_round_bracket_df_for_merge,
      this_round_bracket_df_for_merge,
      on = ['next_round_desc', 'next_round_slot'],
      how = "inner",
      suffixes=('_1', '_2')
      )

    this_round_matchups = (this_round_matchups[
      this_round_matchups["round_slot_1"] < this_round_matchups["round_slot_2"]
      ].
      drop(["round_slot_1", "round_slot_2"], axis = 1).
      rename(columns = {
        'next_round_desc': 'round_desc',
        'next_round_slot': 'round_slot'
        }).
      sort_values(["round_slot"]).
      reset_index(drop = True)
      )

    # Original (non-parallel) calls to Gemini to get winner of each matchup
    this_round_matchups['gemini_matchup_analyis'] = this_round_matchups.apply(
      lambda row: get_matchup_winner_from_gemini(row.to_dict(), BRACKET_THEME,
        BRACKET_ADVANCEMENT_SPECIAL_CRITERIA, GROUNDING_SOURCE,
        PROVIDED_MATERIAL_FILE_URI),
      axis = 1
      )

    this_round_matchups['gemini_matchup_analyis_dict'] = (
        this_round_matchups['gemini_matchup_analyis'].apply(json.loads))

    for col in ['winning_item', 'winning_margin', 'explanation']:
      this_round_matchups[col] = (
          this_round_matchups['gemini_matchup_analyis_dict'].apply(
              lambda x: x[col])
          )

    this_round_matchups['losing_item'] = np.where(
      this_round_matchups['winning_item'] == this_round_matchups['item_1'],
      this_round_matchups['item_2'],
      this_round_matchups['item_1']
      )

    this_round_results = this_round_matchups[['round_desc', 'round_slot',
      'winning_item', 'losing_item', 'winning_margin', 'explanation']]

    round_by_round_results = round_by_round_results + [this_round_results]

    print(f"Done with Round {this_round}")

    # Updating round count before next iteration of loop
    this_round = this_round - 1

    # Reassigning df to help w/ "this round" becoming "prev_round" in next round
    prev_round_results = this_round_results.copy()

    del this_round_results

all_round_results = pd.concat(round_by_round_results, ignore_index=True)

In [ ]:
#@title Prepare Bracket Advancement Data for Showing on Bracket
bracket_advancement_with_info = pd.merge(
  pd.merge(
    # Merge results with bracket rounds to get more info on previous round
    pd.merge(
      all_round_results[['round_desc', 'round_slot', 'winning_item',
          'losing_item', 'winning_margin', 'explanation']].
        assign(
          prev_round_desc = lambda x: (x['round_desc'] + 1)
          ).
        rename(columns = {"winning_item": "item"}),
      BRACKET_ROUNDS[['round_desc', 'round_name']].
        rename(columns = {
          "round_desc": "prev_round_desc",
          "round_name": "prev_matchup_round"
          }),
      on = 'prev_round_desc',
      how = 'left'
      ),
    # Merge with bracket info on winning item to use in bracket
    bracket_generation_with_info[['item', 'short_name', 'original_rank', 'seed',
      'overall_slot', 'display_text']],
    on = 'item',
    how = 'left'
    ),
    # Merge with bracket info on losing item to use in bracket text
    bracket_generation_with_info[['item', 'seed']].
      rename(columns = {"item": "losing_item", "seed": "losing_seed"}),
    on = 'losing_item',
    how = 'left'
  )

bracket_advancement_with_info['hover_text'] = (bracket_advancement_with_info.
  apply(lambda row:
    f"{row['seed']} {row['item']}<br>"
    f"{row['prev_matchup_round']}: defeated {row['losing_seed']} "
    f"{row['losing_item']} ({row['winning_margin']})<br>"
    f"{row['explanation']}"
    ,
    axis=1
    )
  )

display(bracket_advancement_with_info)

In [ ]:
#@title Show Generated Bracket with Advancement
bracket_with_winners_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_advancement_with_info,
  bracket_with_items_viz,
  trim_bracket_viz_vertically = TRIM_BRACKET_VISUAL_VERTICALLY
  )

bracket_with_winners_viz = bracket_with_winners_viz_and_dfs['bracket_viz']

display(bracket_with_winners_viz)

## Add Celebratory Image for Champion to Bracket
---

In [ ]:
#@title Functions to Create Celebratory Image for Champion and Add to Bracket
bracket_champion = (bracket_advancement_with_info[
  bracket_advancement_with_info['round_desc'] == 0]['item'].iloc[0])

def get_bracket_champion_image(bracket_theme, num_items, bracket_champion):
    try:
        bracket_champion_imagen_prompt = f"""
        Celebratory photo of {bracket_champion} as the champion,
        emerging from a {num_items}-item bracket of {bracket_theme}
        """
        bracket_champion_image_gen = GENAI_CLIENT.models.generate_images(
          model=IMAGEN_MODEL_ID,
          prompt=bracket_champion_imagen_prompt,
          config=types.GenerateImagesConfig(
            number_of_images=1,
            aspect_ratio="1:1",
            person_generation="ALLOW_ADULT",
            safety_filter_level="BLOCK_LOW_AND_ABOVE",
            enhance_prompt=True,
            include_rai_reason=True,
            include_safety_attributes=True
            )
        )

        bracket_champion_image = (bracket_champion_image_gen.
          generated_images[0].image)

        display_image(bracket_champion_image)

    except:
        bracket_champion_imagen_prompt = f"""
        Celebratory photo of a single item from the theme of {bracket_theme}
        as the champion, emerging from a {num_items}-item bracket
        """
        print(
          "Image generation failed with original prompt, trying more generic:"
          f"{bracket_champion_imagen_prompt}"
          )

        bracket_champion_image_gen = GENAI_CLIENT.models.generate_images(
          model=IMAGEN_MODEL_ID,
          prompt=bracket_champion_imagen_prompt,
          config=types.GenerateImagesConfig(
            number_of_images=1,
            aspect_ratio="1:1",
            person_generation="ALLOW_ADULT",
            safety_filter_level="BLOCK_LOW_AND_ABOVE",
            enhance_prompt=True,
            include_rai_reason=True,
            include_safety_attributes=True
            )
          )

        bracket_champion_image = (bracket_champion_image_gen.
          generated_images[0].image)

        display_image(bracket_champion_image)

    return bracket_champion_image

def add_bracket_champion_image_to_bracket(exisiting_bracket_viz, bracket_df,
    bracket_champion_image, num_items):

    # Get some image parameters related to num items
    BrCo = get_bracket_viz_config(num_items)

    bracket_champion_image._pil_image.save("bracket_champion.png")

    bracket_champion_image_base64 = get_image_base64("bracket_champion.png")

    bracket_df_champ_info = bracket_df[bracket_df['round_desc'] == 0].iloc[0]

    bracket_viz_with_champ_image = copy.deepcopy(exisiting_bracket_viz)

    bracket_viz_with_champ_image.add_layout_image(
      dict(
        source='data:image/png;base64,' + bracket_champion_image_base64,
        xref="x",
        yref="y",
        x=bracket_df_champ_info['rect_mid_x'],
        y=bracket_df_champ_info['rect_bottom_y'] - BrCo["image_space_vs_min_y"],
        sizex=BrCo["image_width"],
        sizey=BrCo["image_height"],
        xanchor="center",
        yanchor="top"
        )
      )

    return(bracket_viz_with_champ_image)

In [ ]:
#@title Add Champion Celebratory Image to Bracket
# Wrap image generation and addition to bracket in case it fails
try:
    bracket_champion_image = get_bracket_champion_image(
      bracket_theme = BRACKET_THEME,
      num_items = NUM_ITEMS,
      bracket_champion = bracket_champion
      )

    clear_output()

    bracket_with_champ_image_viz = add_bracket_champion_image_to_bracket(
      bracket_with_winners_viz,
      bracket_with_winners_viz_and_dfs['bracket_dfs']['bracket_df'],
      bracket_champion_image,
      NUM_ITEMS
      )

except:
    print("Image generation and addition to bracket failed, proceed without it")
    bracket_with_champ_image_viz = bracket_with_winners_viz

display(bracket_with_champ_image_viz)

## Get Bracket Results (Including Gemini Summary) as Text
---

In [ ]:
#@title Functions to Get Text Pieces from Bracket Results
def get_bracket_items_text_from_df(bracket_df):
    bracket_items_text = "##Bracket Items and Descriptions##\n\n"

    for region in sorted(bracket_df['region_name'].unique()):
      bracket_items_text += f"###{region}###\n\n"
      for index, row in (
        bracket_df[bracket_df['region_name'] == region].
        sort_values("seed").
        iterrows()
        ):
        bracket_items_text += (
          f"**{row['seed']}) {row['item']}** (Orig Rk #"
          f"{row['original_rank']}): {row['description']}\n\n"
          )
      bracket_items_text += "\n"

    return(bracket_items_text)

def get_bracket_results_text_from_df(bracket_results_df):
    bracket_df_grouped_results = bracket_results_df.groupby(
      ['prev_round_desc', 'prev_matchup_round'])

    # Store group keys in a list
    bracket_df_group_keys = list(bracket_df_grouped_results.groups.keys())

    # Combine results into a single string
    round_by_round_results_text = '##Round-by-Round Results##\n\n'

    # Iterate through group keys in reverse order
    for round_name in reversed(bracket_df_group_keys):
        round_data = bracket_df_grouped_results.get_group(round_name)
        round_by_round_results_text += f"\n\n###{round_name[1]}###\n"
        for index, row in round_data.iterrows():
            round_by_round_results_text += (
              f"**{row['seed']} {row['item']} def. {row['losing_seed']} "
              f"{row['losing_item']}** ({row['winning_margin']}): "
              f"{row['explanation']}\n\n"
              )

    return(round_by_round_results_text)

def get_bracket_gemini_summary(bracket_title, bracket_info_and_results_text):

    bracket_summary_system_instruction = types.Part.from_text(text="""
    You are an expert at summarizing the results from a competition among
    various items that can be organized into a bracket form. In your summaries,
    focus on the items that advanced further more and the overall champion that
    emerged at the end of the bracket the most.
    """)

    bracket_summary_config = types.GenerateContentConfig(
      temperature = 1.0,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [bracket_summary_system_instruction],
      response_modalities = ["TEXT"]
      )

    bracket_summary_prompt = types.Part.from_text(text=f"""
    Summarize the {bracket_title} using the information and results below in
    about 250 words.
    Bracket info and results:
    {bracket_info_and_results_text}
    """)

    bracket_summary_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=bracket_summary_prompt,
      config=bracket_summary_config
      )

    bracket_summary_text = ("##Bracket Results Summary from Gemini##\n\n"
      f"{bracket_summary_response.text}")

    return(bracket_summary_text)

def get_all_bracket_text(bracket_title, bracket_df, bracket_results_df):

    bracket_items_text = get_bracket_items_text_from_df(bracket_df)

    round_results_text = get_bracket_results_text_from_df(bracket_results_df)

    bracket_items_results_text = (f"{bracket_items_text}\n\n\n"
      f"{round_results_text}")

    bracket_gemini_summary = get_bracket_gemini_summary(
        bracket_title, bracket_items_results_text)

    all_combined_text = (
      f"#{bracket_title}#\n\n"
      f"{bracket_gemini_summary}\n\n\n"
      f"{bracket_items_results_text}"
      )

    return(all_combined_text)

In [ ]:
#@title Get All Bracket Text (Gemini Summary, Items, Round-by-Round Results)
all_bracket_text = get_all_bracket_text(
  bracket_title = BRACKET_TITLE,
  bracket_df = bracket_generation_with_info,
  bracket_results_df = bracket_advancement_with_info
  )

display(Markdown(all_bracket_text))

## Download Final Bracket and Related Outputs
----

In [ ]:
#@title Download Bracket as Static PNG Image
png_output_file_name = f'{BRACKET_TITLE}.png'

pio.write_image(bracket_with_champ_image_viz, png_output_file_name)

files.download(png_output_file_name)

In [ ]:
#@title Download Bracket as Interactive HTML File
html_output_file_name = f'{BRACKET_TITLE}.html'

pio.write_html(bracket_with_champ_image_viz, file=html_output_file_name)

files.download(html_output_file_name)

In [ ]:
#@title Download Bracket Summary Text as PDF
bracket_text_file_name = f"{BRACKET_TITLE}.pdf"

bracket_text_html_content = markdown.markdown(all_bracket_text)
HTML(string=bracket_text_html_content).write_pdf(bracket_text_file_name)

clear_output()

files.download(bracket_text_file_name)